In [2]:
from torchsummary import summary
import pandas as  pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy
import math
from torch.nn import Parameter
#from keras.utils import to_categorical
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score,f1_score
from imblearn.over_sampling import SMOTE
import torch.nn.functional as F
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import roc_auc_score
#acc = accuracy_score(test_Y1, predicted1)

In [52]:
a = pd.read_csv('1iso.csv')
b = pd.read_csv('2iso.csv')
c = pd.read_csv('3iso.csv')
d = pd.read_csv('4iso.csv')
e = pd.read_csv('5iso.csv')

In [53]:
# 使用concat函数拼接数据框
result = pd.concat([a, b,c,d,e])
search = result.loc[result['predicted'] == 0]
print(result)

           id  ec_label  predicted
0      Q9Z1J2        32         32
1    A8MQ27-2        32         32
2    A8MQ27-3       115        115
3    D0EZM8-2       115        115
4    D0EZM8-3       102        102
..        ...       ...        ...
400    Q9Y697        16         16
401  Q9Y6K1-2        66         66
402  Q9Y6M4-5        11         11
403  Q9Y6P5-2        87         87
404  Q9Y6W6-2        66         66

[2027 rows x 3 columns]


In [54]:
acc = accuracy_score(result['ec_label'], result['predicted'])
f1ma = f1_score(result['ec_label'], result['predicted'], average='macro')
print(acc)
print(f1ma)

0.9684262456832757
0.884030150149894


In [55]:
result['level0_ec'] = 1
result['predicted_y'] = 1

In [56]:
# 将predicted_x为0的行的predicted_x的值改为888
result['predicted_x'] = result['predicted']
result.loc[result['predicted_y'] == 0, 'predicted_x'] = 888

In [57]:
print(result)

           id  ec_label  predicted  level0_ec  predicted_y  predicted_x
0      Q9Z1J2        32         32          1            1           32
1    A8MQ27-2        32         32          1            1           32
2    A8MQ27-3       115        115          1            1          115
3    D0EZM8-2       115        115          1            1          115
4    D0EZM8-3       102        102          1            1          102
..        ...       ...        ...        ...          ...          ...
400    Q9Y697        16         16          1            1           16
401  Q9Y6K1-2        66         66          1            1           66
402  Q9Y6M4-5        11         11          1            1           11
403  Q9Y6P5-2        87         87          1            1           87
404  Q9Y6W6-2        66         66          1            1           66

[2027 rows x 6 columns]


In [58]:
acc = accuracy_score(result['ec_label'], result['predicted_x'])
f1ma = f1_score(result['ec_label'], result['predicted_x'], average='macro')
print(acc)
print(f1ma)

0.9684262456832757
0.884030150149894


In [45]:
dict_ec_label = np.load('ec_dict_level4.npy', allow_pickle=True).item()
for key, value in dict_ec_label.items():
    if value == '2.7.2.8':
        print(key, value)

160 2.7.2.8


In [59]:
dict_ec_label = np.load('iso_ec_dict.npy', allow_pickle=True).item()
#t['ec'] = t.ec_label.apply(lambda x: dict_ec_label.get(x))
result['predicted_level4'] = result.predicted_x.apply(lambda x: dict_ec_label.get(x))
result['ec_number'] = result.ec_label.apply(lambda x: dict_ec_label.get(x))
print(result)

           id  ec_label  predicted  level0_ec  predicted_y  predicted_x  \
0      Q9Z1J2        32         32          1            1           32   
1    A8MQ27-2        32         32          1            1           32   
2    A8MQ27-3       115        115          1            1          115   
3    D0EZM8-2       115        115          1            1          115   
4    D0EZM8-3       102        102          1            1          102   
..        ...       ...        ...        ...          ...          ...   
400    Q9Y697        16         16          1            1           16   
401  Q9Y6K1-2        66         66          1            1           66   
402  Q9Y6M4-5        11         11          1            1           11   
403  Q9Y6P5-2        87         87          1            1           87   
404  Q9Y6W6-2        66         66          1            1           66   

    predicted_level4 ec_number  
0           2.3.2.26  2.3.2.26  
1           2.3.2.26  2.3.2.26  


In [61]:
# 用空字符串填充缺失值
result['ec_number'] = result['ec_number'].fillna('0.0.0.0')
result['predicted_level4'] = result['predicted_level4'].fillna('0.0.0.0')
# 分割 EC 号的每一层
result['ec_first_layer'] = result['ec_number'].apply(lambda x: x.split('.')[0])
result['prediction_first_layer'] = result['predicted_level4'].apply(lambda x: x.split('.')[0])
# 比较原始 EC 号和预测 EC 号的第一层是否相同
# 将predicted_x为0的行的rediction_first_layer的值改为0.0.0.0
result.loc[result['predicted_y'] == 0, 'rediction_first_layer'] = '0.0.0.0'
result['first_layer_correct'] = result['ec_first_layer'] == result['prediction_first_layer']
# 计算预测正确的比例
first_layer_accuracy = result['first_layer_correct'].mean()
print(f"第一层的预测正确率为: {first_layer_accuracy * 100:.2f}%")
f1 = f1_score(result['ec_first_layer'], result['prediction_first_layer'], average='weighted')
print(f"F1 分数（加权）: {f1:.4f}")

第一层的预测正确率为: 98.57%
F1 分数（加权）: 0.9857


In [62]:
# 分割 EC 号的第二层
result['ec_second_layer'] = result['ec_number'].apply(lambda x: x.split('.')[1] if len(x.split('.')) > 1 else '')
result['prediction_second_layer'] = result['predicted_level4'].apply(lambda x: x.split('.')[1] if len(x.split('.')) > 1 else '')

# 比较原始 EC 号和预测 EC 号的第一层和第二层是否都相同
result['first_and_second_layer_correct'] = (result['ec_first_layer'] == result['prediction_first_layer']) & (result['ec_second_layer'] == result['prediction_second_layer'])

# 计算第一层和第二层都正确的准确率
first_and_second_layer_accuracy = result['first_and_second_layer_correct'].mean()
print(f"第一层和第二层都正确的预测准确率为: {first_and_second_layer_accuracy * 100:.2f}%")
# 合并第一层和第二层的 EC 号作为新的标签
result['ec_combined'] = result['ec_first_layer'] + '.' + result['ec_second_layer']
result['prediction_combined'] = result['prediction_first_layer'] + '.' + result['prediction_second_layer']
# 计算 F1 分数
f1_combined = f1_score(result['ec_combined'], result['prediction_combined'], average='weighted')
print(f"第一层和第二层都正确的 F1 分数（加权）: {f1_combined:.4f}")

第一层和第二层都正确的预测准确率为: 98.08%
第一层和第二层都正确的 F1 分数（加权）: 0.9805


In [63]:
# 分割 EC 号的第三层
result['ec_third_layer'] = result['ec_number'].apply(lambda x: x.split('.')[2] if len(x.split('.')) > 2 else '')
result['prediction_third_layer'] = result['predicted_level4'].apply(lambda x: x.split('.')[2] if len(x.split('.')) > 2 else '')

# 比较原始 EC 号和预测 EC 号的第一层、第二层和第三层是否都相同
result['first_second_third_layer_correct'] = (result['ec_first_layer'] == result['prediction_first_layer']) & \
                                             (result['ec_second_layer'] == result['prediction_second_layer']) & \
                                             (result['ec_third_layer'] == result['prediction_third_layer'])

# 计算第一层、第二层和第三层都正确的准确率
first_second_third_layer_accuracy = result['first_second_third_layer_correct'].mean()
print(f"第一层、第二层和第三层都正确的预测准确率为: {first_second_third_layer_accuracy * 100:.2f}%")
# 合并第一层、第二层和第三层的 EC 号作为新的标签
result['ec_combined_layers'] = result['ec_first_layer'] + '.' + result['ec_second_layer'] + '.' + result['ec_third_layer']
result['prediction_combined_layers'] = result['prediction_first_layer'] + '.' + result['prediction_second_layer'] + '.' + result['prediction_third_layer']
# 计算 F1 分数
f1_combined_layers = f1_score(result['ec_combined_layers'], result['prediction_combined_layers'], average='weighted')
print(f"第一层、第二层和第三层都正确的 F1 分数（加权）: {f1_combined_layers:.4f}")

第一层、第二层和第三层都正确的预测准确率为: 96.99%
第一层、第二层和第三层都正确的 F1 分数（加权）: 0.9695


In [51]:
print(result)

           id  ec_label  predicted  level0_ec  predicted_y  predicted_x  \
0      Q9Z1J2        32         32          1            1           32   
1    A8MQ27-2        32         32          1            1           32   
2    A8MQ27-3       115        115          1            1          115   
3    D0EZM8-2       115        115          1            1          115   
4    D0EZM8-3       102        102          1            1          102   
..        ...       ...        ...        ...          ...          ...   
400    Q9Y697        16         16          1            1           16   
401  Q9Y6K1-2        66         66          1            1           66   
402  Q9Y6M4-5        11         11          1            1           11   
403  Q9Y6P5-2        87         87          1            1           87   
404  Q9Y6W6-2        66         66          1            1           66   

    predicted_level4 ec_number ec_first_layer prediction_first_layer  ...  \
0           2.3.2.26  

In [106]:
result.to_csv('result_final_1121.csv',index=False)